In [ ]:
import modules.m11_load_data, modules.m12_transform_data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

**Import Data**

In [ ]:
df_users, df_movies, df_ratings = modules.m12_transform_data.transform_data(only_ratings_set=False)

**Explore Customers data**

In [ ]:
df_users.head()

In [ ]:
df_users.info()

In [ ]:
print(
    'Num. of unique customers :' , df_users['userId'].nunique()
    , '\nUserId range             :', df_users['userId'].min(), '-', df_users['userId'].max()
)

# distribution of gender variable
gender_distr = df_users.groupby(by = ['gender']).size().reset_index().rename(columns={0:'size'})
gender_distr['share'] = (gender_distr['size']/gender_distr['size'].sum() ).round(2)
print('\nGender Distribution\n', gender_distr.set_index('gender').sort_values(by=['share'], ascending=[False]) )

# distribution of age variable
age_distr = df_users.groupby(by = ['age']).size().reset_index().rename(columns={0:'size'})
age_distr['share'] = (age_distr['size']/age_distr['size'].sum() ).round(2)
print('\nAge Distribution\n',
      age_distr[['age', 'size', 'share']]\
      .set_index('age').sort_values(by=['size'], ascending=[False])
     )

In [ ]:
age_distr = age_distr[['age', 'size', 'share']].set_index('age').sort_values(by=['age'], ascending=[True]).reset_index()
age_distr['age_bin'] = np.where( age_distr['age'] <= 18, 0,
                               np.where(age_distr['age'] <= 29, 1,
                                        np.where(age_distr['age'] <= 39, 2,
                                                 np.where(age_distr['age'] <= 49, 3,
                                                          np.where(age_distr['age'] <= 59, 4, 5)
                                                         )
                                                )
                                       )
                               )
#sns.barplot(age_distr[['age', 'size', 'share']].set_index('age').sort_values(by=['age'], ascending=[True]).reset_index()
#            , x="age", y="share")
age_distr.groupby(by=['age_bin'])[['size', 'share']].sum()

In [ ]:
# distribution of job variable
job_distr = df_users.groupby(by = ['job']).size().reset_index().rename(columns={0:'size'})
job_distr['share'] = (job_distr['size']/job_distr['size'].sum() ).round(2)
print('\nJob Distribution\n',
      job_distr[['job', 'size', 'share']]\
      .set_index('job').sort_values(by=['size'], ascending=[False])
     )

In [ ]:
# distribution of zip code variable
print( 'Num. of unique zip codes' , df_users['zip_code'].nunique() )

zip_distr = df_users.groupby(by = ['zip_code']).size().reset_index().rename(columns={0:'size'})
zip_distr['share'] = (zip_distr['size']/zip_distr['size'].sum() ).round(2)
print('\nZip Code Distribution\n',
      zip_distr.set_index('zip_code').sort_values(by=['size'], ascending=[False])
      )

**Explore Movies' data**

In [ ]:
df_movies.head()

In [ ]:
df_movies.info()

In [ ]:
(df_movies.iloc[:,4:].sum()/df_movies.shape[0]*100).reset_index().sort_values(by=0,ascending=False)

In [ ]:
print(
    'Num. of unique movies :' , df_movies['itemId'].nunique()
    , '\nMovieId range         :', df_movies['itemId'].min(), '-', df_movies['itemId'].max()
)

**Explore Ratings' data**

In [ ]:
df_ratings.head()

In [ ]:
df_ratings.info()

In [ ]:
"""
Does it appear multiple ratings in a movie by the same user?
    >> NO, it is only registered a single rating per movie for all the users
    
How many movies were rated by each user? (distribution incl. average, median, ...)
How many users rated each of the movies? (distribution incl. average, median, ...)

"""

In [ ]:
# Checking whether or not there are records of multiples ratings for the same iteam for each of the users
df_ratings[df_ratings[['userId', 'itemId']].duplicated()].shape[0]

fig, ax =plt.subplots(1,2, figsize=(15,5))
for col,i in zip(['itemId', 'userId'], [0,1]):

    ax[i].set_title(f"Distribution of {col} column")
    sns.histplot(
        data=df_ratings.groupby(by=[col]).size().reset_index().rename(columns={0:'vol_ratings'})['vol_ratings']
        , binwidth = 50, stat = 'count', kde = True, ax = ax[i]
    )
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    
    print( df_ratings.groupby(by=[col]).size().reset_index().rename(columns={0:'vol_ratings'})['vol_ratings'].describe().round() )

In [ ]:
fig, ax =plt.subplots(1,1, figsize=(15,10))
for col,i in zip(['itemId'], [0]):

    ax.set_title(f"Distribution of {col} column")
    sns.histplot(
        data=df_ratings.groupby(by=[col]).size().reset_index().rename(columns={0:'vol_ratings'})['vol_ratings']
        , binwidth = 50, stat = 'percent', kde = True, ax = ax, color= "#FA920C"
    )
    ax.set_facecolor("#000E21")
    #plt.tick_params(top=False, bottom=False, left=False, right=False,
    #            labelleft=False, labelbottom=False)


In [ ]:
df_ratings['date'].min(), df_ratings['date'].max()

In [ ]:
df_ratings[df_ratings['date']>='19980222']

In [ ]:
# bias towards high ratings
aux = df_ratings.groupby('rating').size().reset_index().rename(columns={0:'size'})
aux['share'] = (aux['size']/aux['size'].sum() *100).round(1)
aux

In [ ]:
aux = df_ratings.groupby('date').size().reset_index().rename(columns={0:'vol_ratings'})

# add date columns
aux['date_d'] = pd.to_datetime(aux['date'], format='%Y%m%d')
aux['yearweek'] = aux['date_d'].dt.strftime('%Y') + aux['date_d'].dt.isocalendar().week.astype(str).str.zfill(2)
aux['yearweek'] = np.where(aux['yearweek'] == '199701', '199801', aux['yearweek'])

display(aux.head())
display(aux.groupby('yearweek')['vol_ratings'].sum().reset_index().head())

aux_2 = aux.groupby('yearweek')['vol_ratings'].sum().reset_index()

# Showing every 5th label
tick_positions = aux_2['yearweek'][::5]  # Select every 5th week
tick_labels = aux_2['yearweek'][::5]
plt.figure(figsize=(10, 5))
sns.lineplot(data=aux_2, x="yearweek", y="vol_ratings")
plt.xticks(ticks=tick_positions, labels=tick_labels, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
aux.sort_values('vol_ratings', ascending = False)

In [ ]:
# most popular movies
display(
    df_movies[df_movies['itemId'].isin(
        df_ratings\
            .groupby('itemId').size().reset_index().rename(columns={0:'size'})\
            .sort_values(by='size', ascending = False).head(3)['itemId'].values
    )]
)

# hotest movies
aux = df_ratings\
    .groupby('itemId').agg({'rating' : ['mean', 'size']}).sort_values([('rating', 'mean')], ascending=False)
display(
    df_movies[df_movies['itemId'].isin(
        aux[aux['rating']['size'] >= aux['rating']['size'].mean()].head(3).index
    )]
)

In [ ]:
df_ratings[df_ratings['itemId'].isin([168, 317, 407])]\
    .groupby('itemId').agg({'rating' : ['mean', 'size']}).sort_values([('rating', 'mean')], ascending=False)

In [ ]:
aux = df_ratings.copy()
aux['hour'] = df_ratings['time'].str.slice(start=0, stop=2)
aux = aux.groupby('hour').size().reset_index().rename(columns={0:'size'} )
aux['share'] = (aux['size']/aux['size'].sum()*100).round(2)
aux.sort_values('size')

In [ ]:
# Convert hour '24' to '00' for correct ordering and plotting
aux['hour'] = aux['hour'].replace('24', '00')

# Sort DataFrame by 'hour' to ensure correct plotting order
aux = aux.sort_values(by='hour')

# Convert 'hour' back to '24' at display time in labels if needed
aux['hour_label'] = aux['hour'].replace('00', '24')
#display(aux)
# Create the radar chart
labels = aux['hour']
stats = aux['share']
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()

# Complete the loop
stats = np.concatenate((stats,[stats[0]]))
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
ax.fill(angles, stats, 'b', alpha=0.1)
ax.plot(angles, stats, 'b', linewidth=2)  # Line plot
ax.set_xticks(angles)
#ax.set_xticklabels(labels)

# Display settings
ax.set_title('Hourly Share of Records', size=20, color='b', y=1.1)

plt.show()

In [ ]:
aux = df_ratings.merge(
    df_movies[['itemId']+list(df_movies.columns[4:])]
    , on = 'itemId', how='left'
)
genre_dict = []
for col_genre in list(df_movies.columns[4:]):
    genre_dict_i = {}
    aux_i = aux[aux[col_genre]==1]
    genre_dict_i['genre'] = col_genre
    genre_dict_i['vol_ratings'] = aux_i.shape[0]
    genre_dict_i['avg_rating'] = aux_i['rating'].mean().round(2)

    for r in [1,2,3,4,5]:
        genre_dict_i['vol_rat'+str(r)] = aux_i[aux_i['rating']==r].shape[0]
        genre_dict_i['share_rat'+str(r)] = np.round(aux_i[aux_i['rating']==r].shape[0]/aux_i.shape[0]*100)
    genre_dict.append(genre_dict_i)

aux = pd.DataFrame(genre_dict)
aux['share_ratings'] = (aux['vol_ratings']/df_ratings.shape[0]*100).round()
aux.sort_values(by='vol_ratings', ascending=False)

In [ ]:
aux = df_ratings.copy()

# add date columns
aux['date_d'] = pd.to_datetime(aux['date'], format='%Y%m%d')
aux['yearweek'] = aux['date_d'].dt.strftime('%Y') + aux['date_d'].dt.isocalendar().week.astype(str).str.zfill(2)
aux['yearweek'] = np.where(aux['yearweek'] == '199701', '199801', aux['yearweek'])

aux = aux.merge(
    df_movies[['itemId']+list(df_movies.columns[4:])]
    , on = 'itemId', how='left'
)

df_aux = []
for col_genre in list(df_movies.columns[4:]):
    
    genre_dict_i = {}
    aux_i = aux[aux[col_genre]==1]
    
    aux_is = aux_i.groupby('yearweek').agg({'rating': ['count', 'mean']}).round(2).droplevel(0, axis=1).reset_index()\
                .merge(
                    aux_i.groupby(['yearweek', 'rating']).size().reset_index().rename(columns={0:'size'})\
                        .pivot(index='yearweek', columns='rating', values='size').fillna(0).reset_index()
                    , on='yearweek', how = 'left'
                    ).rename(columns = {1:'vol_r1', 2:'vol_r2', 3:'vol_r3', 4:'vol_r4', 5:'vol_r5'})
    aux_is['genre'] = col_genre
    aux_is = aux_is[ [aux_is.columns[-1]] + list(aux_is.columns[:-1])]
    df_aux.append(aux_is)

df_aux = pd.concat(df_aux)[['genre', 'yearweek', 'count', 'mean', 'vol_r1', 'vol_r2', 'vol_r3', 'vol_r4', 'vol_r5']].fillna(0)
for col_r in ['vol_r1', 'vol_r2', 'vol_r3', 'vol_r4', 'vol_r5']:
    df_aux['share_'+col_r[-2:]] = (df_aux[col_r]/df_aux['count']*100).round()

# filter by most popular genres
genres_list = ['Drama']
# ['Sci-Fi', 'Adventure', 'Romance', 'Thriller', 'Action', 'Comedy', 'Drama']
df_aux = df_aux[df_aux['genre'].isin(genres_list)]

# cum sum
df_aux['count_cumsum'] = df_aux.groupby('genre')['count'].cumsum()
df_aux = df_aux.merge(
    df_aux.groupby('genre')['count'].sum().reset_index().rename(columns={'count':'total_vol_rat'})
    , how = 'left', on='genre')
df_aux['count_cumper'] = (df_aux['count_cumsum']/df_aux['total_vol_rat']*100).round()
df_aux.drop(columns=['total_vol_rat'], inplace=True)

display(df_aux)

# Showing every 5th label
tick_positions = df_aux['yearweek'].unique()[::5]  # Select every 5th week
tick_labels = df_aux['yearweek'].unique()[::5]
plt.figure(figsize=(10, 5))
sns.lineplot(data=df_aux, x="yearweek", y="mean")
ax2 = plt.twinx()
sns.lineplot(data=df_aux, x="yearweek", y="share_r1", ax = ax2, color='orange')
plt.xticks(ticks=tick_positions, labels=tick_labels, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df_aux.to_csv('test.csv')

In [ ]:
# general overview
aux = df_ratings.groupby('userId').agg({'rating':['count', 'mean', 'std']}).round(2).droplevel(0, axis=1).reset_index()
# rating overview
aux2 = df_ratings.groupby(['userId', 'rating']).size().reset_index().rename(columns={0:'size'})\
        .pivot(index='userId', columns='rating', values='size').fillna(0).reset_index()\
        .rename(columns={1:'vol_r1', 2:'vol_r2', 3:'vol_r3', 4:'vol_r4', 5:'vol_r5'})

# overview + rating overview
aux = aux.merge(aux2, how='left', on='userId')
del aux2
for col_r in ['vol_r1', 'vol_r2', 'vol_r3', 'vol_r4', 'vol_r5']:
    aux['share'+col_r[-3:]] = (aux[col_r]/aux['count']*100).round()

# genre overview
list_df_genre =[
    df_ratings.merge( df_movies[['itemId']+list(df_movies.columns[4:])], on = 'itemId', how='left')
]
for col_genre in list(df_movies.columns[5:]):
    aux2_i = list_df_genre[0][list_df_genre[0][col_genre]==1]
    
    aux2_i1 = aux2_i.groupby('userId').agg({'rating':['count', 'mean', 'std']}).round(2).droplevel(0, axis=1).reset_index().fillna(0)
    aux2_i2 = aux2_i.groupby(['userId', 'rating']).size().reset_index().rename(columns={0:'size'})\
        .pivot(index='userId', columns='rating', values='size').fillna(0).reset_index()\
        .rename(columns={1:'vol_r1', 2:'vol_r2', 3:'vol_r3', 4:'vol_r4', 5:'vol_r5'})
    aux2_i1 = aux2_i1.merge(aux2_i2, how='outer', on='userId')
    del aux2_i2, aux2_i
    for col_r in ['vol_r1', 'vol_r2', 'vol_r3', 'vol_r4', 'vol_r5']:
        aux2_i1['share'+col_r[-3:]] = (aux2_i1[col_r]/aux2_i1['count']*100).round()

    aux2_i1.columns = col_genre+'_'+aux2_i1.columns
    aux2_i1.rename(columns={col_genre+'_'+'userId' :'userId'}, inplace=True)
    list_df_genre.append(aux2_i1)

    #display(aux2_i1)

df_final = list_df_genre[1]
for df in list_df_genre[2:]:
    df_final = df_final.merge(df, how='outer', on = 'userId')

aux = aux.merge(df_final.fillna(0), how='left', on='userId')

In [ ]:
aux[aux['share_r5']>=30]

In [ ]:
df_user_rating = (aux['share_r4']+aux['share_r5']
                 ).reset_index().rename(columns={0:'share'})
df_user_rating[df_user_rating['share']>=50]

In [ ]:
df_ratings['rating'].mean()